In [8]:
import pandas as pd
import re
import nltk
from nltk.tag import pos_tag
from typing import List
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import numpy as np

In [9]:
from gensim.models.word2vec import Word2Vec
import gensim

In [13]:
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True) 

In [49]:
dialogues = pd.read_csv('../code/emo_context_train.txt', sep="\t")
class_counts = dialogues["label"].value_counts()

# Print the class counts
print("class counts: ",class_counts)
dialogues = dialogues[dialogues["label"]!="others"]
labels = dialogues.label
cols = ["turn1", "turn2", "turn3"]
dialogues["combined"] = dialogues[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
dialogues["combined"] = dialogues["combined"].apply(str.lower).apply(word_tokenize).apply(lambda x: " ".join(x))
dialogues["seq_len"] = dialogues["combined"].apply(len)
print(dialogues.columns)

max_len = dialogues["seq_len"].max()
input_text = dialogues.combined.values
tokenized = []
for sen in input_text:
    tokenized.append(sen.split())


class counts:  others    14948
angry      5506
sad        5463
happy      4243
Name: label, dtype: int64
Index(['id', 'turn1', 'turn2', 'turn3', 'label', 'combined', 'seq_len'], dtype='object')
15212


In [15]:
import sklearn
from sklearn.model_selection import train_test_split

In [16]:
X = tokenized
y = labels

In [22]:
train_data, val_data, train_labels, val_labels = train_test_split(tokenized, labels, test_size=0.2, random_state=42)

In [65]:
X_train, X_test, y_train, y_test = train_test_split(tokenized, y, test_size=0.2, random_state=16)

In [72]:
train_vectors = []
for sentence in X_train:
    sentence_vec = []
    for word in sentence:
        if word in word2vec_model:
            sentence_vec.append(word2vec_model[word])
    train_vectors.append(np.mean(sentence_vec, axis=0))
    
test_vectors = []
for sentence in X_test:
    sentence_vec = []
    for word in sentence:
        if word in word2vec_model:
            sentence_vec.append(word2vec_model[word])
    test_vectors.append(np.mean(sentence_vec, axis=0))
    
print(len(test_vectors))

3043


In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

In [71]:
logisticRegr = LogisticRegression(max_iter=10000)

logR = LogisticRegression()
logR.fit(train_vectors, y_train)

/Users/nina/miniconda3/envs/ml/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [73]:
predictionsLog = logR.predict(test_vectors)

In [74]:
print("accuracy: ",accuracy_score(y_test, predictionsLog))
print("F1 score: ",f1_score(y_test, predictionsLog, average="macro"))

accuracy:  0.8008544199802826
F1 score:  0.8005721462946648


In [76]:
from sklearn.metrics import classification_report
 
print(classification_report(y_test, predictionsLog))

              precision    recall  f1-score   support

       angry       0.79      0.82      0.81      1100
       happy       0.82      0.78      0.80       830
         sad       0.80      0.79      0.80      1113

    accuracy                           0.80      3043
   macro avg       0.80      0.80      0.80      3043
weighted avg       0.80      0.80      0.80      3043

